In [14]:
import random

literki = ['ą', 'ć', 'ę', 'ł', 'ń', 'ó', 'ś', 'ź', 'ż']
litery = ['a', 'c', 'e', 'l', 'n', 'o', 's', 'z', 'z']


def letter_size_error(sentence):
    words = sentence.split(' ')
    n = len(words)
    r = random.randint(0, n-1)
    words[r] = words[r].upper()
    return ' '.join(words)


def missing_letter_error(sentence):
    words = sentence.split(' ')
    n = len(words)
    r = random.randint(0, n-1)
    m = len(words[r])
    while m < 2:
        r -= 1
        m = len(words[r])

    words[r] = words[r][:-1]
    return ' '.join(words)


def polish_filter(x):
    if x in literki:
        return litery[literki.index(x)]
    else:
        return x


def polish_error(sentence):
    return ''.join(map(polish_filter, sentence))


with open('reviews.txt', 'r') as inputF, open('reviews_a.txt', 'w') as outputF:
    for line in inputF:
        sentiment, review = line.split('#')
        review = review.strip()
        review = letter_size_error(review)
        review = missing_letter_error(review)
        review = polish_error(review)
        outputF.write(sentiment + '# ' + review + '\n')

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import pandas as pd


model_name_pap = 'flax-community/papuGaPT2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizerPap = AutoTokenizer.from_pretrained(model_name_pap)
modelPap = AutoModelForCausalLM.from_pretrained(model_name_pap).to(device)

PROMPT = 'Powiedz to innymi słowami, nie zmieniając wydźwięku zdania: '


def rewrite(sentence):
    inputs = tokenizerPap(PROMPT + sentence,
                          return_tensors='pt').to(device)
    outputs = modelPap.generate(**inputs,
                                max_length=len(inputs['input_ids'][0]) + 20,
                                num_return_sequences=1,
                                do_sample=True,
                                eos_token_id=tokenizerPap.encode('.')[0],
                                pad_token_id=tokenizerPap.encode('.')[0]
                                )
    return tokenizerPap.decode(outputs[0], skip_special_tokens=True)[len(PROMPT) + len(sentence):]


print(rewrite('To jest bardzo dobra książka.'))

with open('reviews.txt', 'r') as inputF, open('reviews_b.txt', 'w') as outputF:
    for line in inputF:
        sentiment, review = line.split('#')
        review = review.strip()
        review = rewrite(review)
        review = review.replace('\n', ' ')
        outputF.write(sentiment + '# ' + review + '\n')

 Jest genialna, napisana z niezwykłą precyzją, zaskakująca.
